In [2]:
import os

from pandas.io.parquet import to_parquet

In [3]:
%pwd


'D:\\HourlyConsumptiomXGboost\\research'

In [4]:
# os.chdir(r"D:\HourlyConsumptiomXGboost")

In [5]:
%pwd

'D:\\HourlyConsumptiomXGboost'

In [18]:
# entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    # source_URL:str
    local_data_file: Path

In [19]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            # source_URL=config.source_URL,
            local_data_file=config.local_data_file1
        )

        return data_ingestion_config

In [21]:
import os
import urllib.request as request
import pandas as pd
from src.mlProject import logger
from src.mlProject.utils.common import get_size,get_mongoData


In [22]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
           
    def initiateDataIngestion(self):
        try:
            data = get_mongoData()
            logger.info("Reading Completed from MongoDB")
            logger.info("Writing data in Parquet file")
            data.to_parquet(self.config.local_data_file1, index=False, engine='fastparquet')
            read_df = pd.read_parquet(self.config.local_data_file1)
            print(read_df)
            
        except Exception as e:
            print(f"Error: {e}")
  

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.initiateDataIngestion()
    # data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-02 12:27:48,051: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-02 12:27:48,054: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-02 12:27:48,060: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-02 12:27:48,062: INFO: common: created directory at: artifacts]
[2023-11-02 12:27:48,064: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-02 12:27:48,066: INFO: common: calling DB configuration]
[2023-11-02 12:27:48,067: INFO: common: Reading data from Mongo DB]


KeyboardInterrupt: 

In [2]:
import os

In [3]:

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/bhattshubham1401/HourlyConsumptionPredN.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="bhattshubham1401"
os.environ["MLFLOW_TRACKING_PASSWORD"]="be6bd2c38bbcc62d1ddea7c7489e2e83e67ec230"

In [1]:
import pandas as pd
import numpy as np

# Your original DataFrame
data = {
    'timestamp': ['2013-06-14 01:22:10', '2013-06-14 01:22:11', '2013-06-14 01:22:12', '2013-06-14 01:22:13', '2013-06-14 01:22:14'],
    'active': [217.259, 225.991, 223.157, 225.118, 221.923],
    'reactive': [-28.605, -29.788, -29.026, -29.511, -29.281],
    'aparent': [219.134, 227.946, 225.037, 227.044, 223.846],
    'frequency': [50.28, 50.28, 50.30, 50.29, 50.29],
    'voltage': [230.94, 231.01, 230.90, 230.93, 230.98],
    'PF': [0.991, 0.991, 0.992, 0.992, 0.991],
    'current': [0.949, 0.987, 0.975, 0.983, 0.969],
}

df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Harmonics calculation function
def calculate_harmonics(signal):
    n = len(signal)
    fft_result = np.fft.fft(signal)
    positive_frequencies = fft_result[:n//2]
    amplitudes = np.abs(positive_frequencies)
    phases = np.angle(positive_frequencies)
    return np.concatenate([amplitudes, phases])

# Apply harmonics calculation to voltage and current columns
df['voltage_harmonics'] = df['voltage'].apply(calculate_harmonics)
df['current_harmonics'] = df['current'].apply(calculate_harmonics)

# Create the new DataFrame with harmonics and appliances
columns = []
for col in ['current', 'voltage']:
    for i in range(1, len(df[col+'_harmonics'][0])+1):
        columns.append(f'{col}_h{i}')

columns.append('appliance')
new_df = pd.DataFrame(columns=columns)

# Populate the new DataFrame
for index, row in df.iterrows():
    harmonics_row = []
    for col in ['current', 'voltage']:
        harmonics_row.extend(row[f'{col}_harmonics'])
    harmonics_row.append('Your_Appliance_Here')  # Replace with the actual appliance label
    new_df = new_df.append(pd.Series(harmonics_row, index=columns), ignore_index=True)

# Display the new DataFrame
print(new_df)


TypeError: object of type 'float' has no len()